In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE = 'surrogates/psychic_classifier_11.pth'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier

from art.attacks.evasion import FastGradientMethod as FGM

import logging
import pandas as pd
import numpy as np
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
import os
path = os.path.join(os.getcwd(), '..','observation_masks.csv')
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [7]:
kwargs = dict(norm=np.inf,
        eps=0.05,
        num_random_init=0,
        targeted=False,
        minimal=False
        )
attack = FGM(estimator=classifier(
                        model=torch.load(SURROGATE),
                        loss=CrossEntropyLoss(), 
                        nb_classes=11, 
                        input_shape=agent.observation_space.shape,
                        device_type='gpu',
                        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                        ),
            **kwargs
             )

In [8]:
logging.basicConfig(level=logging.WARN)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,
                                                      mask=observation_masks['mask_time'].to_numpy().astype(float),
                                                      time_steps=None)

100%|██████████| 8759/8759 [01:24<00:00, 103.57it/s, ASR=0.299]
c:\users\broda-milian\onedrive - rmc-cmr.ca\documents\citylearn examples\modules\KBMproject\utilities.py:195: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return format_kpis(env), np.array(obs_list), np.array(adv_obs_list), asr


In [9]:
display(kpi)

cost_function
annual_peak_average                      1.000000
carbon_emissions_total                   0.880484
cost_total                               0.796169
daily_one_minus_load_factor_average      1.051436
daily_peak_average                       0.903066
electricity_consumption_total            0.891650
monthly_one_minus_load_factor_average    0.973922
ramping_average                          1.154870
zero_net_energy                          1.109143
Name: District, dtype: float64